In [1]:
import numpy as np

a = 'tgcatcgagaccctacgtgac'
b = 'actagacctagcatcgac'
match = 1
mismatch = 1
gap = 1
def best_matrix(a,b,match,mismatch,gap): #a:sequence1, b:sequence2
    bm = np.zeros((len(a)+1,len(b)+1),int)#best_matrix
    max_score = 0
    max_pos = 0
    for i in range(len(a)+1):
        for j in range(len(b)+1):
            if i == 0 or j == 0:
                bm[i][j] = 0  #Set the gap in the matrix
            else:
                match_score = bm[i - 1, j - 1] + (match if a[i - 1] == b[j - 1] else - mismatch)
                gap1 = bm[i - 1, j] - gap
                gap2 = bm[i, j - 1] - gap
                score = max(match_score, gap1, gap2, 0)
                bm[i, j] = score
                if score > max_score:
                    max_score = score
                    max_pos = (i, j) #max_position 
    print(bm)
    return(bm,max_pos)

#https://tiefenauer.github.io/blog/smith-waterman/         

# test: http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Smith-Waterman


In [2]:
#diagonal: match
#up: gap in b
#left: gap in a
#https://stackoverflow.com/questions/23400317/smith-waterman-algorithm-to-generate-matrix-in-python
def go(bm, x, y): #x,y: position in matrix
    diagonal = bm[x-1][y-1]
    up = bm[x-1][y]
    left = bm[x][y-1]
    if diagonal >= up and diagonal >= left:
        if diagonal != 0:
            return "diagonal"
        else: 
            return 0
    elif up > diagonal and up >= left:
        if up != 0:
            return "up"
        else: 
            return 0
    elif left > diagonal and left > up:
        if left != 0:
            return "left"
        else: 
            return 0
    else:
        print("WRONG")

In [3]:
#https://stackoverflow.com/questions/12666494/how-do-i-decide-which-way-to-backtrack-in-the-smith-waterman-algorithm
def traceback(a, b, bm, start_pos):
    aligned_seq1 = []
    aligned_seq2 = []
    x, y = start_pos
    move = go(bm, x, y)
    while move != 0:
        if move == "diagonal":
            aligned_seq1.append(a[x-1])
            aligned_seq2.append(b[y-1])
            x = x - 1
            y = y - 1
        elif move == "up":
            aligned_seq1.append(a[x-1])
            aligned_seq2.append('-')
            x = x - 1
        else: #move left
            aligned_seq1.append('-')
            aligned_seq2.append(b[y-1])
            y = y - 1

        move = go(bm, x, y)
    
    aligned_seq1.append(a[x - 1])
    aligned_seq2.append(b[y - 1])

    return ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))

In [4]:
def align(a,b, match,mismatch,gap):
    bm, max_pos = best_matrix(a,b, match,mismatch,gap)
    score = bm[max_pos[0]][max_pos[1]]
    a, b = traceback(a, b, bm, max_pos)
    print ("seq1 = ", a, ", seq2 = ", b, ", score =" , score)

In [256]:
align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', 1, 1, 1)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 2 1 0 1 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 1 0 1 3 2 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 2 4 3 2 1 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 1 1 3 5 4 3 2]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 2 4 6 5 4]
 [0 1 0 0 1 0 2 1 0 0 1 0 0 1 1 3 5 7 6]
 [0 0 0 0 0 2 1 1 0 0 0 2 1 0 0 2 4 6 6]
 [0 1 0 0 1 1 3 2 1 0 1 1 1 2 1 1 3 5 5]
 [0 0 2 1 0 0 2 4 3 2 1 0 2 1 1 2 2 4 6]
 [0 0 1 1 0 0 1 3 5 4 3 2 1 1 0 2 1 3 5]
 [0 0 1 0 0 0 0 2 4 4 3 2 3 2 1 1 1 2 4]
 [0 0 0 2 1 0 0 1 3 5 4 3 2 2 3 2 1 1 3]
 [0 1 0 1 3 2 1 0 2 4 6 5 4 3 2 2 1 2 2]
 [0 0 2 1 2 2 1 2 1 3 5 5 6 5 4 3 2 1 3]
 [0 0 1 1 1 3 2 1 1 2 4 6 5 5 4 3 4 3 2]
 [0 0 0 2 1 2 2 1 0 2 3 5 5 4 6 5 4 3 2]
 [0 0 0 1 1 2 1 1 0 1 2 4 4 4 5 5 6 5 4]
 [0 1 0 0 2 1 3 2 1 0 2 3 3 5 4 4 5 7 6]
 [0 0 2 1 1 1 2 4 3 2 1 2 4 4 4 5 4 6 8]]
seq1 =  agacccta-cgt-gac , seq2 =  agacc-tagcatcgac , score = 8


In [257]:
align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', 1, 1, 2)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 2 0 0 1 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 1 0 0 3 1 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 1 4 2 0 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 1 0 2 5 3 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 3 6 4 2]
 [0 1 0 0 1 0 2 0 0 0 1 0 0 1 0 1 4 7 5]
 [0 0 0 0 0 2 0 1 0 0 0 2 0 0 0 0 2 5 6]
 [0 1 0 0 1 0 3 1 0 0 1 0 1 1 0 0 0 3 4]
 [0 0 2 0 0 0 1 4 2 0 0 0 1 0 0 1 0 1 4]
 [0 0 1 1 0 0 0 2 5 3 1 0 1 0 0 1 0 0 2]
 [0 0 1 0 0 0 0 1 3 4 2 0 1 0 0 1 0 0 1]
 [0 0 0 2 0 0 0 0 1 4 3 1 0 0 1 0 0 0 0]
 [0 1 0 0 3 1 1 0 0 2 5 3 1 1 0 0 0 1 0]
 [0 0 2 0 1 2 0 2 1 0 3 4 4 2 0 1 0 0 2]
 [0 0 0 1 0 2 1 0 1 0 1 4 3 3 1 0 2 0 0]
 [0 0 0 1 0 0 1 0 0 2 0 2 3 2 4 2 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1 1 1 2 2 3 3 1 0]
 [0 1 0 0 1 0 2 0 0 0 1 0 0 2 1 1 2 4 2]
 [0 0 2 0 0 0 0 3 1 0 0 0 1 0 1 2 0 2 5]]
seq1 =  gcatcga , seq2 =  gcatcga , score = 7


In [9]:
align('tgcatcgagaact', 'actagacctagcatcg',2, 2, 2)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  2  0  0  0  0  2  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  2  0  0  0  0  2]
 [ 0  0  2  0  0  0  0  2  2  0  0  0  4  2  0  2  0]
 [ 0  2  0  0  2  0  2  0  0  0  2  0  2  6  4  2  0]
 [ 0  0  0  2  0  0  0  0  0  2  0  0  0  4  8  6  4]
 [ 0  0  2  0  0  0  0  2  2  0  0  0  2  2  6 10  8]
 [ 0  0  0  0  0  2  0  0  0  0  0  2  0  0  4  8 12]
 [ 0  2  0  0  2  0  4  2  0  0  2  0  0  2  2  6 10]
 [ 0  0  0  0  0  4  2  2  0  0  0  4  2  0  0  4  8]
 [ 0  2  0  0  2  2  6  4  2  0  2  2  2  4  2  2  6]
 [ 0  2  0  0  2  0  4  4  2  0  2  0  0  4  2  0  4]
 [ 0  0  4  2  0  0  2  6  6  4  2  0  2  2  2  4  2]
 [ 0  0  2  6  4  2  0  4  4  8  6  4  2  0  4  2  2]]
seq1 =  gcatcg , seq2 =  gcatcg , score = 12
